The code is from https://github.com/HarshSulakhe/3DMNIST-PyTorch

# Import Packages

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import h5py
from utils.plot3D import *
from utils.utils import *

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(device)

cuda:0


# Load the vectors 

In [3]:
with h5py.File('/home/asabuncuoglu13/dataset/mnist3d/full_dataset_vectors.h5', "r") as hf:    
    X_train = hf["X_train"][:]
    targets_train = hf["y_train"][:]
    X_test = hf["X_test"][:] 
    targets_test = hf["y_test"][:]
    
X_train = rgb_data_transform(X_train)
X_test = rgb_data_transform(X_test)

# Create Datasets and Dataloaders

In [4]:
train_x = torch.from_numpy(X_train).float()
train_y = torch.from_numpy(targets_train).long()
test_x = torch.from_numpy(X_test).float()
test_y = torch.from_numpy(targets_test).long()

batch_size = 100

train_ds = torch.utils.data.TensorDataset(train_x,train_y)
test_ds = torch.utils.data.TensorDataset(test_x,test_y)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size = batch_size, shuffle = False)

# CNN and Linear Models

In [47]:
import torch.nn.init as init
import math

class AttentionConv3d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, bias=False):
        super(AttentionConv3d, self).__init__()
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.groups = groups

        assert self.out_channels % self.groups == 0, "out_channels should be divided by groups. (example: out_channels: 40, groups: 4)"

        self.rel_h = nn.Parameter(torch.randn(out_channels // 2, 1, 1, kernel_size, 1), requires_grad=True)
        self.rel_w = nn.Parameter(torch.randn(out_channels // 2, 1, 1, 1, kernel_size), requires_grad=True)

        self.key_conv = nn.Conv3d(in_channels, out_channels, kernel_size=3, bias=bias)
        self.query_conv = nn.Conv3d(in_channels, out_channels, kernel_size=3, bias=bias)
        self.value_conv = nn.Conv3d(in_channels, out_channels, kernel_size=3, bias=bias)

        self.reset_parameters()

    def forward(self, x):
        batch, channels, height, width, d = x.size()
        print("x size:", x.size())

        padded_x = F.pad(x, (self.padding, self.padding, self.padding, self.padding, self.padding, self.padding))
        print("padded x size:", padded_x.size())
        
        q_out = self.query_conv(x)
        print("q_out:", q_out.size())
        
        k_out = self.key_conv(padded_x)
        print("k_out:", k_out.shape)
        
        v_out = self.value_conv(padded_x)
        print(v_out.size())
        
        k_out = k_out.contiguous().view(batch, self.groups, self.out_channels // self.groups, height, width, d, -1)
        v_out = v_out.contiguous().view(batch, self.groups, self.out_channels // self.groups, height, width, d, -1)

        q_out = q_out.view(batch, self.groups, self.out_channels // self.groups, height, width, d, 1)

        out = q_out * k_out
        out = F.softmax(out, dim=-1)
        out = torch.einsum('bncdhwk,bncdhwk -> bncdhw', out, v_out).view(batch, -1, height, width, d)
        
        return out

    def reset_parameters(self):
        init.kaiming_normal_(self.key_conv.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.value_conv.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.query_conv.weight, mode='fan_out', nonlinearity='relu')

        init.normal_(self.rel_h, 0, 1)
        init.normal_(self.rel_w, 0, 1)

In [48]:
class CNNModel(nn.Module):

    def __init__(self):
        super(CNNModel,self).__init__()
        self.conv1 = nn.Sequential(
            AttentionConv3d(3, 8, kernel_size=5, padding=1, groups=2),
            nn.BatchNorm3d(8),
            nn.ReLU(),
        )        
        #self.conv1 = nn.Conv3d(3,8,3)
        self.conv2 = nn.Conv3d(8,32,3)
        self.conv3 = nn.Conv3d(32,32,3)
        # self.conv4 = nn.Conv3d(64,128,3)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool3d(4)
        # self.pool2 = nn.MaxPool3d(2)
        # self.linear1 = nn.Linear(128*8,128)
        self.linear2 = nn.Linear(32,10)
        self.batchnorm = nn.BatchNorm3d(32)
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.conv3(x)
        # x = self.conv4(x)
        # x = self.pool1(x)
        x = self.batchnorm(x)
        x = x.view(x.size()[0],-1)
        # x = self.linear1(x)
        # x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

# Define Hyperparameters

In [49]:
num_epochs = 160

model = CNNModel()
model = model.cuda()

error = nn.CrossEntropyLoss()

learning_rate = 0.00005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the Model

In [50]:
train_loss_list = []
test_loss_list = []
train_accuracy_list = []
test_accuracy_list = []
for epoch in range(num_epochs):
    model.train()
    running_acc = 0
    train_loss = 0
    for i, (images, labels) in enumerate(train_loader):     
        train = Variable(images.permute(0,4,1,2,3)).cuda()
        labels = Variable(labels).cuda()
        optimizer.zero_grad()
        outputs = model(train)
        loss = error(outputs, labels)
        train_loss+=loss.data
        train_accuracy = accuracy_score(torch.argmax(outputs,axis = 1).cpu().numpy(),labels.cpu().numpy())
        running_acc+=train_accuracy*len(images)
        loss.backward()
        optimizer.step()
#         scheduler.step()
    running_acc/=len(train_ds)
    train_loss/=5
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            test = Variable(images.permute(0,4,1,2,3)).cuda()
            outputs = model(test).detach()
            test_loss = error(outputs,labels.cuda())
            test_accuracy = accuracy_score(torch.argmax(outputs,axis = 1).cpu().numpy(),labels.cpu().numpy())
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss.data)
    test_accuracy_list.append(test_accuracy)
    train_accuracy_list.append(train_accuracy)
    

    print('Epoch: {}  Train Loss:{:.2f}, Test Loss:{:.2f}, Train Accuracy:{:.2f}, Test Accuracy:{:.2f}'.format(epoch, train_loss,test_loss,train_accuracy,test_accuracy))

x size: torch.Size([100, 3, 16, 16, 16])
padded x size: torch.Size([100, 3, 18, 18, 18])
q_out: torch.Size([100, 8, 14, 14, 14])
k_out: torch.Size([100, 8, 16, 16, 16])
torch.Size([100, 8, 16, 16, 16])


RuntimeError: shape '[100, 2, 4, 16, 16, 16, 1]' is invalid for input of size 2195200

# Test the Model

In [ ]:
model.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        test = data[0].to(device)
        labels = data[1].to(device)
        outputs = model(test).detach()
        test_loss = error(outputs,labels.cuda())
        test_accuracy = accuracy_score(torch.argmax(outputs,axis = 1).cpu().numpy(),labels.cpu().numpy())
print(test_accuracy)
torch.save(model.state_dict,'./CNN_5e-5.pth')

# Generate Plots

In [ ]:
plt.plot(train_loss_list,label = 'train')
plt.plot(test_loss_list,label = 'test')
plt.legend()
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.title("Loss vs Number of epochs")
plt.show()

In [ ]:
plt.plot(train_accuracy_list,color = "red",label = 'train')
plt.plot(test_accuracy_list,color = "green",label = 'test')
plt.legend()
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Number of epochs")
plt.show()

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/3dmnist')
test = Variable(images.permute(0,4,1,2,3)).cuda()
writer.add_graph(model,test)
# writer.close()